In [55]:
#Get the speech to text command. 
import sounddevice as sd
import numpy as np
import whisper
from scipy.io.wavfile import write

# Step 1: Record audio from microphone
fs = 44100  # Sample rate
seconds = 30  # Duration of recording
print("Recording... Speak now!")
recording = sd.rec(int(seconds * fs), samplerate=fs, channels=1, dtype=np.int16)
sd.wait()
wav_filename = "recorded_audio.wav"
write(wav_filename, fs, recording)
print(f"Recording saved as '{wav_filename}'.")

# Step 2: Transcribe the audio using Whisper
model = whisper.load_model("base")  # Use "tiny", "small", "medium", or "large" as needed
result = model.transcribe(wav_filename)

# Step 3: Print the transcription
print("Transcription:")
print(result["text"])
message=result["text"]

Recording... Speak now!
Recording saved as 'recorded_audio.wav'.


/Users/hjiang/miniforge3/envs/ai-agent/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription:
 Add a card called AI agent under to-do list. For the description, say research RAG models.


In [56]:
#get list information and creates a dataframe for the rag model to reference. leaves description blank for now.

import requests
import pandas as pd

# Replace with your board ID
board_id = "5f1743793b184e31ee7d27f9"  # Example board ID

# Trello API URL for getting lists on a board
url = f"https://api.trello.com/1/boards/{board_id}/lists"

# Query parameters (API key and token)
query = {
    'key': 'd200138058c063cbb4ccd1c46bb572a0',  # Your API Key
    'token': 'ATTA2e7844b7cbb2557cf79ef8cd019d9f64f0fc77840bc4410149219a6f447c56b4D33239C4'  # Your API Token
}

# Make the request
response = requests.request("GET", url, params=query)

# Check if the request was successful
if response.status_code == 200:
    lists_data = response.json()
    
    # Convert JSON to DataFrame (no description included)
    df = pd.DataFrame(lists_data)
    
    # Select relevant columns (id and name)
    df = df[['id', 'name',]]  # Only keep id and name columns
    
    # Print the DataFrame
    print(df)
    
    # Creating a text context for the LLM (RAG model)
    context = "\n".join([f"ID: {row['id']}, Name: {row['name']}" for _, row in df.iterrows()])
    
    print("==== Context for RAG Model ====")
    print(context)
else:
    print(f"Error: {response.status_code}, {response.text}")

# Print the response for debugging
print(response.status_code)  # Check the status code (200 means success)
print(response.json())  # Print the JSON response (lists on the board)


                         id          name
0  5f1743793b184e31ee7d27fb  Things To Do
1  5f1743793b184e31ee7d27fc         Doing
2  5f1743793b184e31ee7d27fd          Done
==== Context for RAG Model ====
ID: 5f1743793b184e31ee7d27fb, Name: Things To Do
ID: 5f1743793b184e31ee7d27fc, Name: Doing
ID: 5f1743793b184e31ee7d27fd, Name: Done
200
[{'id': '5f1743793b184e31ee7d27fb', 'name': 'Things To Do', 'closed': False, 'color': None, 'idBoard': '5f1743793b184e31ee7d27f9', 'pos': 65535, 'subscribed': False, 'softLimit': None, 'type': None, 'datasource': {'filter': False}}, {'id': '5f1743793b184e31ee7d27fc', 'name': 'Doing', 'closed': False, 'color': None, 'idBoard': '5f1743793b184e31ee7d27f9', 'pos': 131071, 'subscribed': False, 'softLimit': None, 'type': None, 'datasource': {'filter': False}}, {'id': '5f1743793b184e31ee7d27fd', 'name': 'Done', 'closed': False, 'color': None, 'idBoard': '5f1743793b184e31ee7d27f9', 'pos': 196607, 'subscribed': False, 'softLimit': None, 'type': None, 'datasource': 

In [66]:
# Initialize ChatModel for Ollama
# Initialize ChatModel for Ollama
async def main():
    llm = ChatModel.from_name("ollama:granite3.1-dense:8b")
    workflow = AgentWorkflow(name="Smart assistant")

    workflow.add_agent(
        name="ListRetriever",
        role="A smart assistant who identifies the correct Trello list.",
        instructions="Match the user's description to one of the available Trello lists. Return both the ID and name.",
        llm=llm,
    )

    # Example of user input
    user_input = message

    # Use the RAG model to find the best match by passing the DataFrame to the model
    # Convert DataFrame to text representation to give the LLM context (only including ID and Name)
    context = "\n".join([f"ID: {row['id']}, Name: {row['name']}" for _, row in df.iterrows()])

    response = await workflow.run(
        inputs=[
            AgentWorkflowInput(
                prompt=f"Given the following list of Trello lists:\n{context}\n\nUser input: '{user_input}'\n\nWhich Trello list should the card be added to? Provide the ID and name.",
                expected_output="Return the ID and name of the corresponding Trello list.",
            ),
        ]
    ).on(
        "success",
        lambda data, event: print(
            f"\n-> Step '{data.step}' has been completed with the following outcome.\n\n{data.state.final_answer}"
        ),
    )

    print("==== Final Answer ====")
    print(response.result.final_answer)

# Instead of asyncio.run(), simply use await in a running event loop (for Jupyter or similar environments)
await main()


-> Step 'ListRetriever' has been completed with the following outcome.

The card should be added to the "Things To Do" list. The ID is 5f1743793b184e31ee7d27fb and the name is "Things To Do".
==== Final Answer ====
The card should be added to the "Things To Do" list. The ID is 5f1743793b184e31ee7d27fb and the name is "Things To Do".


In [ ]:
#so far the list retriever and only retrieve the list id and name. we need it to get the description in the future

In [67]:


# Function to make a POST request (e.g., add a new card to Trello)
def post_to_trello(list_id: str, card_name: str) -> str:
    url = "https://api.trello.com/1/cards"  # Example Trello API URL to create a new card
    query = {
        'key': 'd200138058c063cbb4ccd1c46bb572a0',  # Your API Key
        'token': 'ATTA2e7844b7cbb2557cf79ef8cd019d9f64f0fc77840bc4410149219a6f447c56b4D33239C4',  # Your API Token
        'idList': list_id,
        'name': card_name,  # Name of the card to be added
    }

    # Sending POST request to Trello API to create the card
    response = requests.post(url, params=query)
    if response.status_code == 200:
        return f"Card '{card_name}' has been successfully added to the list."
    else:
        return f"Failed to add the card. Error: {response.status_code} - {response.text}"

# Initialize ChatModel for Ollama
async def main():
    llm = ChatModel.from_name("ollama:granite3.1-dense:8b")
    workflow = AgentWorkflow(name="Smart assistant")

    workflow.add_agent(
        name="TrelloAssistant",
        role="A smart assistant who adds cards to Trello lists.",
        instructions="Based on user input, identify the list and card name, then post a new card to Trello.",
        llm=llm,
    )

    # Example of user input (asking to add a card to a list)
    user_input = "Add a new card called 'New Task' to the 'Things To Do' list."

    # Extract the Trello list name and the card name from user input
    list_name = "Things To Do"
    card_name = "New Task 2"

    # Retrieve the list ID from the DataFrame (using the list name)
    list_id = df.loc[df['name'] == list_name, 'id'].values[0]

    # Call the POST request to add the card to Trello
    post_response = post_to_trello(list_id, card_name)

    # Provide response to the user
    response = await workflow.run(
        inputs=[
            AgentWorkflowInput(
                prompt=f"User asked: '{user_input}'. How should I respond after adding a new card to the Trello list?",
                expected_output="Provide a confirmation message or an error message if something goes wrong.",
            ),
        ]
    ).on(
        "success",
        lambda data, event: print(f"\n-> Step '{data.step}' has been completed with the following outcome.\n\n{data.state.final_answer}")
    )

    print("==== Final Answer ====")
    print(post_response)

# Instead of asyncio.run(), simply use await in a running event loop (for Jupyter or similar environments)
await main()



-> Step 'TrelloAssistant' has been completed with the following outcome.

I have successfully added a new card titled "New Task" to your 'Things To Do' list on Trello. You can now view this task in your board. If you need further assistance, please let me know.
==== Final Answer ====
Card 'New Task 2' has been successfully added to the list.


In [ ]:
#this successfully added a card on trello.